In [1]:

import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
from scipy import spatial
import torch
import spacy
en_nlp = spacy.load('en')

In [3]:
import wikipedia
print(wikipedia.summary("Wikipedia"))

Wikipedia ( (listen),  (listen) WIK-ih-PEE-dee-ə) is a multilingual, web-based, free encyclopedia based on a model of openly editable and viewable content, a wiki. It is the largest and most popular general reference work on the World Wide Web, and is one of the most popular websites by Alexa rank. It is owned and supported by the Wikimedia Foundation, a non-profit organization which operates on money it receives from donors.Wikipedia was launched on January 15, 2001, by Jimmy Wales and Larry Sanger. Sanger coined its name, as a portmanteau of wiki and "encyclopedia". Initially an English-language encyclopedia, versions in other languages were quickly developed. With 5,756,837 articles, the English Wikipedia is the largest of the more than 290 Wikipedia encyclopedias. Overall, Wikipedia comprises more than 40 million articles in 301 different languages and by February 2014 it had reached 18 billion page views and nearly 500 million unique visitors per month.In 2005, Nature published a 

In [9]:
search = wikipedia.search('Mercury')

In [154]:
wikipedia.page(search[1]).content

'Freddie Mercury (born Farrokh Bulsara; 5 September 1946 – 24 November 1991) was a British singer-songwriter and record producer, best known as the lead vocalist of the rock band Queen. He was known for his flamboyant stage persona and four-octave vocal range. Mercury wrote numerous hits for Queen, including "Bohemian Rhapsody", "Killer Queen", "Somebody to Love", "Don\'t Stop Me Now",  "Crazy Little Thing Called Love", and "We Are the Champions". He led a solo career while performing with Queen, and occasionally served as a producer and guest musician for other artists.\nMercury was born of Parsi descent in Zanzibar, and grew up there and in India before moving with his family to Middlesex, England, in his late teens. He formed Queen in 1970 with guitarist Brian May and drummer Roger Taylor. Mercury died in 1991 at age 45 due to complications from AIDS, having confirmed the day before his death that he had contracted the disease.\nIn 1992, Mercury was posthumously awarded the Brit Awa

In [ ]:
try:
...     mercury = wikipedia.summary("Mercury")
... except wikipedia.exceptions.DisambiguationError as e:
...     print e.options

In [21]:
for res in search[0:2]:
    try :
        print(wikipedia.summary(res))
    except wikipedia.exceptions.DisambiguationError as e:
        continue

Frederick "Freddie" Mercury (born Farrokh Bulsara; 5 September 1946 – 24 November 1991) was a British singer-songwriter and record producer, best known as the lead vocalist of the rock band Queen. He was known for his flamboyant stage persona and four-octave vocal range. Mercury wrote numerous hits for Queen, including "Bohemian Rhapsody", "Killer Queen", "Somebody to Love", "Don't Stop Me Now",  "Crazy Little Thing Called Love", and "We Are the Champions". He led a solo career while performing with Queen, and occasionally served as a producer and guest musician for other artists.
Mercury was born of Parsi descent in Zanzibar, and grew up there and in India before moving with his family to Middlesex, England, in his late teens. He formed Queen in 1970 with guitarist Brian May and drummer Roger Taylor. Mercury died in 1991 at age 45 due to complications from AIDS, having confirmed the day before his death that he had contracted the disease.
In 1992, Mercury was posthumously awarded the 

## Try to create embedding using google universal encoder (Not done yet)

## Convert Json to Pandas Dataframe

In [23]:
# read data
train = pd.read_json("data/train-v1.1.json")

valid = pd.read_json("data/dev-v1.1.json")

In [43]:
train.shape, valid.shape

((442, 2), (48, 2))

In [44]:
train.head(3)

,data,version
0,"{'title': 'University_of_Notre_Dame', 'paragra...",1.1
1,"{'title': 'Beyoncé', 'paragraphs': [{'context'...",1.1
2,"{'title': 'Montana', 'paragraphs': [{'context'...",1.1


In [45]:
train.iloc[1,0]['paragraphs'][0]

{'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'qas': [{'answers': [{'answer_start': 269, 'text': 'in the late 1990s'}],
   'question': 'When did Beyonce start becoming popular?',
   'id': '56be85543aeaaa14008c9063'},
  {'answers': [{'answer_start': 207, 'text': 'singing and dancing'}],
   'question': 'What areas did Beyonce compet

In [73]:
valid.iloc[1,0]['paragraphs'][0]

{'context': 'One of the most famous people born in Warsaw was Maria Skłodowska-Curie, who achieved international recognition for her research on radioactivity and was the first female recipient of the Nobel Prize. Famous musicians include Władysław Szpilman and Frédéric Chopin. Though Chopin was born in the village of Żelazowa Wola, about 60 km (37 mi) from Warsaw, he moved to the city with his family when he was seven months old. Casimir Pulaski, a Polish general and hero of the American Revolutionary War, was born here in 1745.',
 'qas': [{'answers': [{'answer_start': 188, 'text': 'Nobel Prize'},
    {'answer_start': 188, 'text': 'Nobel Prize'},
    {'answer_start': 188, 'text': 'Nobel Prize'}],
   'question': 'What was Maria Curie the first female recipient of?',
   'id': '5733a5f54776f41900660f45'},
  {'answers': [{'answer_start': 517, 'text': '1745'},
    {'answer_start': 517, 'text': '1745'},
    {'answer_start': 517, 'text': '1745'}],
   'question': 'What year was Casimir Pulask

In [ ]:
def getContext:
    

In [24]:
# construct relative table
# to save time, use only 1% of orginial data 
contexts = []
questions = []
answers_text = []
answers_start = []
for i in range((int)(train.shape[0]/100)):
    topic = train.iloc[i,0]['paragraphs']
    for sub_para in topic:
        for q_a in sub_para['qas']:
            questions.append(q_a['question'])
            answers_start.append(q_a['answers'][0]['answer_start'])
            answers_text.append(q_a['answers'][0]['text'])
            contexts.append(sub_para['context'])   
df = pd.DataFrame({"context":contexts, "question": questions, "answer_start": answers_start, "text": answers_text})

In [79]:
df.shape[0]

1314

In [109]:
df['search'] =df['question']

In [155]:
def getWikiContext(df):
#     i = 0
#     for q in df['question'][500:501]:
#         if i % 10 == 0:
    for i in range(0,df.shape[0] - 1214):
        print(i)
        q = df['question'][i]
        text = nltk.word_tokenize(q)
        tags = nltk.pos_tag(text)
        newquestion = "";
        for tag in tags:
            if tag[1] in noun_tags:
                newquestion += tag[0] + " "
        if newquestion == "":
            newquestion = q
#         print(newquestion)
        search = wikipedia.search(newquestion)
        df['search'][i] = newquestion
        df['wiki_context'][i] = None
        for res in search:
#             print(res)
            if df['wiki_context'][i] is None:
                try :
#                     print(wikipedia.summary(res))
                    df['wiki_context'][i] = wikipedia.page(res).content
#                     print(df['wiki_context'][i])
                except wikipedia.exceptions.DisambiguationError as e:
                    continue
            else:
                break
        if df['wiki_context'][i] is None:
            df['wiki_context'][i] = "No wikipedia search result found"
        i = i + 1
#     return df

In [159]:
getWikiContext(df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [168]:
df['wiki_context'][53]

'This list of Cornell University alumni includes notable graduates, non-graduate former students, and current students of Cornell University, an Ivy League university located in Ithaca, New York. Cornell counted 245,027 living alumni as of August 2008. Its alumni constitute 25 recipients of National Medal of Science and National Medal of Technology and Innovation combined, 33 MacArthur Fellows, 34 Marshall Scholars and 31 Rhodes Scholars, 231 elected members of the National Academy of Sciences, 174 elected members of the National Academy of Engineering, 180 plus heads of higher learning institutions in the United States and around the world, and Cornell is the only university with three female winners of unshared Nobel Prizes among its graduates (Pearl S. Buck, Barbara McClintock, and Toni Morrison). Many alumni maintain university ties through Homecoming\'s reunion weekend, through Cornell Magazine, and through the Cornell Club of New York. In 2005, Cornell ranked #3 nationwide for gi

In [51]:
noun_tags = ['NN','NNP','NNS','NNPS']

In [113]:
print(df.iloc[509,1])

Who released the information about Beyoncé's performance for the Libyan ruler?


In [123]:
df.iloc[0:100,:]

,context,question,answer_start,text,wiki_context,search
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,Our Lady of Lourdes is a Roman Catholic title ...,Virgin Mary Lourdes France
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,University of Notre Dame's Main Administration...,front Notre Dame Main Building
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,The Basilica of the Sacred Heart in Notre Dame...,Basilica Sacred heart Notre Dame structure
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,The Grotto of Our Lady of Lourdes is located a...,Grotto Notre Dame
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,University of Notre Dame's Main Administration...,top Main Building Notre Dame
5,"As at most other universities, Notre Dame's st...",When did the Scholastic Magazine of Notre dame...,248,September 1876,The Notre Dame Fighting Irish football team is...,Scholastic Magazine Notre dame begin
6,"As at most other universities, Notre Dame's st...",How often is Notre Dame's the Juggler published?,441,twice,The University of Notre Dame du Lac (or simply...,Notre Dame Juggler
7,"As at most other universities, Notre Dame's st...",What is the daily student paper at Notre Dame ...,598,The Observer,The Notre Dame Fighting Irish football team is...,student paper Notre Dame
8,"As at most other universities, Notre Dame's st...",How many student news papers are found at Notr...,126,three,The Michigan–Notre Dame football rivalry is an...,student news papers Notre Dame
9,"As at most other universities, Notre Dame's st...",In what year did the student paper Common Sens...,908,1987,The University of Notre Dame du Lac (or simply...,year student paper Common Sense publication No...


In [59]:
for q in df['question'][11:12]:
    res = wikipedia.search(q)
    text = nltk.word_tokenize(q)
    tags = nltk.pos_tag(text)
    newquestion = "";
    for tag in tags:
        if tag[1] in noun_tags:
            newquestion += tag[0] + " "
    print(newquestion)
    print(wikipedia.search(newquestion))
    print("question is \n"  + q)
    print("search result is " )
#     print(res)
    print(wikipedia.search('the primary seminary of the Congregation of the Holy Cross'))

seminary Congregation Holy Cross 
['Congregation of Holy Cross', 'Congregation of the Holy Spirit', 'Basil Moreau', 'Passionists', 'Marianites of Holy Cross', 'List of Catholic seminaries', 'Holy Cross College, New Zealand', 'Holy Cross School (New Orleans)', 'John Dew (cardinal)', 'Catholic Church in Nigeria']
question is 
What is the primary seminary of the Congregation of the Holy Cross?
search result is 
['Congregation of Holy Cross', 'Congregation of the Holy Spirit', 'Holy Cross School (New Orleans)', 'Passionists', 'Basil Moreau', 'Congregation of Jesus and Mary', 'Knights of the Altar', 'List of religious orders in the Roman Catholic Archdiocese of New York', 'Congregation of the Servants of the Paraclete', 'Neocatechumenal Way']


In [76]:
questions

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [77]:
answers_text[0]

'Saint Bernadette Soubirous'

In [78]:
questions[1]

'What is in front of the Notre Dame Main Building?'

In [79]:
answers_text[1]

'a copper statue of Christ'

In [49]:
len(contexts)

1314

In [50]:
questions[0]

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [145]:
df.iloc[0:100,:].to_csv("data/train_wiki.csv", index = None)

## Create dictionary of sentence embeddings for faster computation

In [169]:
# number of paragraphs
paras = list(df["wiki_context"][0:100].drop_duplicates().reset_index(drop= True))

In [129]:
paras

['Our Lady of Lourdes is a Roman Catholic title of the Blessed Virgin Mary venerated in honour of the Marian apparitions that reportedly occurred in 1858 in the vicinity of Lourdes in France.  The first of these is the apparition of 11 February 1858, when 14-year old Bernadette Soubirous told her mother that a "lady" spoke to her in the cave of Massabielle (a kilometre and a half (1 mi) from the town) while she was gathering firewood with her sister and a friend. Similar apparitions of the "Lady" were reported on seventeen occasions that year, until the climax revelation of Our Lady of the Immaculate Conception took place.In 18 January 1862, Pope Pius IX authorized Bishop Bertrand-Sévère Laurence to permit the veneration of the Blessed Virgin Mary in Lourdes. On 3 July 1876, the same Pontiff officially granted a Canonical Coronation to the image that used to be in the courtyard of what is now part of the Rosary Basilica. The image of Our Lady of Lourdes has been widely copied and repro

In [170]:
len(paras)

41

In [171]:
# use blob to separate sentence
blob = TextBlob(" ".join(paras))
sentences = [item.raw for item in blob.sentences]

In [172]:
len(sentences)

7302

In [173]:
# use infersent to encode sentence
infersent = torch.load('InferSent/encoder/infersent.allnli.pickle', map_location=lambda storage, loc: storage)
infersent.set_glove_path("InferSent/dataset/GloVe/glove.840B.300d.txt")

In [174]:
infersent.build_vocab(sentences, tokenize=True)

Found 20034(/22608) words with glove vectors
Vocab size : 20034


In [175]:
# create sentence embedding
dict_embeddings = {}
for i in range(0,len(sentences)):
    if (i % 100 == 0):
        print(i)
    dict_embeddings[sentences[i]] = infersent.encode([sentences[i]], tokenize=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300


In [176]:
questions = list(df["question"][0:100])

In [177]:


len(questions)


100

In [91]:
# create question embedding
for i in range(len(questions)):
    if (i % 100 == 0):
        print(i)
    dict_embeddings[questions[i]] = infersent.encode([questions[i]], tokenize=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


In [178]:
# create question embedding
for i in range(len(questions)):
    if (i % 100 == 0):
        print(i)
    dict_embeddings[questions[i]] = infersent.encode([questions[i]], tokenize=True)

0


In [98]:
for i, key in enumerate(dict_embeddings):
    print(i)
    print("key is " + key)

0
key is Architecturally, the school has a Catholic character.
1
key is Atop the Main Building's gold dome is a golden statue of the Virgin Mary.
2
key is Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".
3
key is Next to the Main Building is the Basilica of the Sacred Heart.
4
key is Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.
5
key is It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.
6
key is At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
7
key is As at most other universities, Notre Dame's students run a number of news media outlets.
8
key is The nine student-run outlets include three newspapers, both a radio and television station, and several magazines and journals.
9
key 

In [179]:
# use d1 and d2 as train and test set
d3 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings) if i % 2 == 0}
d4 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings) if i % 2 == 1}

In [100]:
d1

{'Architecturally, the school has a Catholic character.': array([[ 0.05519996,  0.05013141,  0.04787038, ...,  0.00821209,
         -0.03642813,  0.044685  ]], dtype=float32),
 'Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".': array([[0.11262652, 0.11146841, 0.14750297, ..., 0.00293285, 0.03322018,
         0.06657628]], dtype=float32),
 'Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.': array([[ 0.04149357,  0.0703306 ,  0.03724371, ...,  0.01096805,
         -0.02892281,  0.0428066 ]], dtype=float32),
 'At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.': array([[ 0.04795522,  0.16508998,  0.09383532, ...,  0.05321149,
         -0.01826634,  0.10806957]], dtype=float32),
 'The nine student-run outlets include three newspapers, both a radio and television sta

In [65]:
d2

{"Atop the Main Building's gold dome is a golden statue of the Virgin Mary.": array([[ 0.07475325,  0.11794458,  0.06240867, ...,  0.01915886,
         -0.02436746,  0.10806957]], dtype=float32),
 'Next to the Main Building is the Basilica of the Sacred Heart.': array([[ 0.08010551,  0.11775322,  0.02186233, ...,  0.01656766,
         -0.01024127,  0.04706628]], dtype=float32),
 'It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.': array([[ 0.10776819,  0.0805801 ,  0.10461736, ...,  0.01522135,
         -0.03814263,  0.14945611]], dtype=float32),
 "As at most other universities, Notre Dame's students run a number of news media outlets.": array([[0.09720325, 0.09345725, 0.05466026, ..., 0.08443642, 0.00817084,
         0.02197512]], dtype=float32),
 'Begun as a one-page journal in September 1876, the Scholastic magazine is issued twice monthly and claims to be the oldest continuous collegiate publication in t

In [180]:
# with open('data/dict_embeddings3.pickle', 'wb') as handle:
#     pickle.dump(d3, handle)

In [181]:
# with open('data/dict_embeddings4.pickle', 'wb') as handle:
#     pickle.dump(d4, handle)

In [182]:
del dict_embeddings